In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import features                      # your helper module

plt.style.use("ggplot")              # nicer default look

# --- Load data --------------------------------------------------
DATA  = Path.cwd().parent            # one level up from /notebooks
train = pd.read_csv(DATA / "train.csv")

# --- Feature engineering ---------------------------------------
train = features.add_time_features(train)   # adds hour/weekday/month
train['datetime'] = pd.to_datetime(train['datetime'])  # ensure datetime dtype

train.head()



,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,weekday,month
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,0,5,1
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1,5,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2,5,1
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,3,5,1
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,4,5,1


In [ ]:
# 02_load_features.py
DATA = Path.cwd().parent        # one level up from /notebooks
train = pd.read_csv(DATA / "train.csv")

# basic engineered columns
train = (
    train.pipe(features.add_time_features)
         .pipe(features.add_weather_buckets)
         # .pipe(features.add_rolling_mean, hours=3)   # optional if you engineered lag
)

# target and initial feature set
TARGET = "count"
FEATURES = ["hour", "weekday", "month",
            "temp", "humidity", "windspeed"]   # start simple

X = train[FEATURES]
y = train[TARGET]

print(f"Feature matrix: {X.shape}\nTarget: {y.shape}")
train.head()


In [ ]:
# 03_train_val_split.py
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.20, random_state=42)

print("Train size:", X_train.shape, "Validation size:", X_val.shape)


In [ ]:
# 04_fit_rf.py
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)
rf.fit(X_train, y_train)


In [ ]:
# 05_eval.py
# clip negatives to satisfy log error
preds = np.clip(rf.predict(X_val), 0, None)

rmsle = mean_squared_log_error(y_val, preds, squared=False)
print(f"Baseline RandomForest RMSLE: {rmsle:.4f}")
